<a href="https://colab.research.google.com/github/YaMUdreC/Ml-and-nn-colab/blob/main/%D0%91%D1%83%D0%B4%D0%BA%D0%BE%D0%B2%2C_%D0%9C%D0%9E_2_%D0%BB%D0%B0%D0%B1%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная № 2

Выполнили: Будков Ярослав, Никиточкина Арина, Помогаев Максим

In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.metrics import classification_report, mean_squared_error

## Загрузка данных и разделение на выборки
В этом разделе мы загружаем данные для задачи классификации и регрессии,
а также разделяем их на обучающие и тестовые выборки в соотношении 80/20.


In [ ]:
# Загрузка данных для классификации
heart_data = pd.read_csv('heart.csv')
X_class = heart_data.drop('target', axis=1) # удаляет столбец с именем 'target'
y_class = heart_data['target']

# Загрузка данных для регрессии
ad_data = pd.read_csv('Advertising.csv')
X_reg = ad_data.drop('sales', axis=1)
y_reg = ad_data['sales']

Разделение данных на обучающие и тестовые выборки

In [ ]:
# Классификация
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# Регрессия
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

## Построение модели классификации

В данном разделе мы настраиваем и обучаем модель SGDClassifier с использованием
GridSearchCV.

Рассматриваем различные функции потерь (perceptron, hinge, squared_hinge)
и методы регуляризации (L1, L2, ElasticNet).

Оцениваем качество модели на тестовой выборке, используя метрику Classification Report.

In [ ]:
# Классификатор

# Словарь, в котором определяются гиперпараметры, которые будут использоваться для поиска оптимальных значений при обучении модели
param_grid_class = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'loss': ['perceptron', 'hinge', 'squared_hinge'],
}
# поиск по сетке (Grid Search) для нахождения наилучших гиперпараметров для модели
# max_iter=1000: максимальное количество итераций для обучения.
# tol=1e-3: Условие остановки, при котором процесс обучения будет остановлен, если изменение в функции потерь менее 0.001
# param_grid_class: Передает ранее определенную сетку параметров для поиска.
# cv=5: данные будут разбиты на 5 частей, и модель будет обучаться и оцениваться на каждой из этих частей
grid_class = GridSearchCV(SGDClassifier(max_iter=1000, tol=1e-3), param_grid_class, cv=5)


# Обучает классификатор с использованием поиска по сетке на тренировочных данных.
# X_train_class: Признаки (входные данные), используемые для обучения.
# y_train_class: Целевые значения (выходные данные), которые модель должна предсказать.
#Этот процесс включает в себя перебор всех комбинаций гиперпараметров из param_grid_class и оценку каждой модели
grid_class.fit(X_train_class, y_train_class)

# Оценка модели
best_class_model = grid_class.best_estimator_ # Сохраняет наилучшую модель
y_pred_class = best_class_model.predict(X_test_class) # Использует наилучшую модель для предсказания классов на тестовых данных
print("Classification Report:")
print(classification_report(y_test_class, y_pred_class)) # отчет о качестве классификации

Classification Report:
              precision    recall  f1-score   support

           0       0.48      1.00      0.64        29
           1       0.00      0.00      0.00        32

    accuracy                           0.48        61
   macro avg       0.24      0.50      0.32        61
weighted avg       0.23      0.48      0.31        61



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Параметр 'penalty'

Этот параметр определяет тип регуляризации, который будет применяться к модели. Регуляризация — это техника, используемая для уменьшения переобучения модели путем добавления штрафа к величине весов модели.

    l1:
        Это L1-регуляризация (также известная как Lasso-регуляризация).
        Она добавляет сумму абсолютных значений коэффициентов (весов) к функции потерь.
        L1-регуляризация может привести к разреженным решениям, т.е. некоторые коэффициенты могут стать равными нулю, что позволяет автоматически выбирать наиболее важные признаки.

    l2:
        Это L2-регуляризация (также известная как Ridge-регуляризация).
        Она добавляет сумму квадратов коэффициентов к функции потерь.
        L2-регуляризация способствует уменьшению величины весов, но в отличие от L1, она не приводит к разреженным решениям. Это значит, что все признаки остаются в модели, но их влияние уменьшается.

    elasticnet:
        Это комбинация L1 и L2-регуляризаций.
        Она позволяет использовать преимущества обеих техник: можно получать как разреженные решения, так и управлять величиной весов.
        Elastic Net особенно полезен, когда количество признаков значительно превышает количество наблюдений или когда признаки коррелируют друг с другом.



Параметр 'loss'

Этот параметр определяет функцию потерь, которая будет использоваться для оптимизации модели. Функция потерь измеряет, насколько хорошо модель предсказывает целевые значения.

    'perceptron':
        Это функция потерь для перцептрона, простейшего типа линейного классификатора.
        Она обновляет веса модели только тогда, когда происходит ошибка в классификации. Сложность этой функции в том, что она не всегда сходится, если данные не линейно разделимы.

    'hinge':
        Потеря хинжа равна нулю, если предсказанное значение уверенно классифицирует пример, и увеличивается, если предсказание неверное или слишком близко к границе между классами

    'squared_hinge':
        Это модификация функции потерь хинжа. Она отличается тем, что вместо линейной зависимости наказывает квадратом ошибки, что усиливает наказание за большие ошибки

    'squared_error':
        Стандартная функция потерь для регрессии, которая измеряет среднее квадратов разностей между предсказанными и истинными значениями.

    'huber':
        Эта функция потерь сочетает в себе свойства квадратичной ошибки и абсолютной ошибки. Она менее чувствительна к выбросам, чем квадратичная ошибка.

    'epsilon_insensitive':
        Она игнорирует ошибки, которые находятся в пределах определенного порога



## Построение модели регрессии

Здесь мы настраиваем и обучаем модель SGDRegressor, используя GridSearchCV.

Рассматриваем функции потерь (squared_error, huber, epsilon_insensitive) и
методы регуляризации (L1, L2, ElasticNet).

Рассчитываем среднеквадратичную ошибку (MSE) на тестовой выборке.

In [ ]:
# Регрессор

# Параметры для GridSearchCV
param_grid_reg = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'loss': ['squared_error', 'huber', 'epsilon_insensitive'],
}

grid_reg = GridSearchCV(SGDRegressor(max_iter=1000, tol=1e-3), param_grid_reg, cv=5)
grid_reg.fit(X_train_reg, y_train_reg)

# Оценка модели
best_reg_model = grid_reg.best_estimator_
y_pred_reg = best_reg_model.predict(X_test_reg)
mse = mean_squared_error(y_test_reg, y_pred_reg)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 5.726730755578482


Регрессор

Определение гиперпараметров для поиска

param_grid_reg: Словарь, содержащий гиперпараметры для поиска оптимальных значений при обучении модели.
  - **penalty**: ['l1', 'l2', 'elasticnet'] — тип регуляризации для борьбы с переобучением.
  - **loss**: ['squared_error', 'huber', 'epsilon_insensitive'] — функции потерь для регрессии.

Поиск по сетке (Grid Search)

GridSearchCV: Используется для перебора всех комбинаций гиперпараметров из **param_grid_reg**.

  **SGDRegressor**:
  - **max_iter=1000**: Максимальное количество итераций обучения.
  - **tol=1e-3**: Условие остановки, при котором обучение завершится, если изменение функции потерь менее 0.001.
  - **cv=5**: Кросс-валидация на 5 фолдах.



## Минипакетная оптимизация (Minibatch Optimization)

В этом разделе реализуется обучение моделей с использованием минипакетов
(minibatches) через метод partial_fit.
Этот метод позволяет эффективно
обновлять веса модели на ограниченном объеме данных.

Сравниваем результаты классификатора и регрессора, обученных методом минипакетов.


In [ ]:
#(Бонус) Minibatch Optimization

# Minibatch Optimization для классификатора
batch_size = 32
perceptron_mb = SGDClassifier(loss='hinge')  # Используем hinge как пример

for i in range(0, len(X_train_class), batch_size):
    X_batch = X_train_class[i:i + batch_size]
    y_batch = y_train_class[i:i + batch_size]
    perceptron_mb.partial_fit(X_batch, y_batch, classes=np.unique(y_train_class))

# Оценка модели
y_pred_class_mb = perceptron_mb.predict(X_test_class)
print("Classification Report (Minibatch):")
print(classification_report(y_test_class, y_pred_class_mb))

# Minibatch Optimization для регрессора
scaler = StandardScaler()
X_train_reg_scaled = scaler.fit_transform(X_train_reg)
X_test_reg_scaled = scaler.transform(X_test_reg)

regressor_mb = SGDRegressor()

for i in range(0, len(X_train_reg_scaled), batch_size):
    X_batch = X_train_reg_scaled[i:i + batch_size]
    y_batch = y_train_reg[i:i + batch_size]
    regressor_mb.partial_fit(X_batch, y_batch)

# Оценка модели
y_pred_reg_mb = regressor_mb.predict(X_test_reg_scaled)
mse_mb = mean_squared_error(y_test_reg, y_pred_reg_mb)
print(f'Mean Squared Error (Minibatch): {mse_mb}')

Classification Report (Minibatch):
              precision    recall  f1-score   support

           0       0.72      0.90      0.80        29
           1       0.88      0.69      0.77        32

    accuracy                           0.79        61
   macro avg       0.80      0.79      0.79        61
weighted avg       0.80      0.79      0.79        61

Mean Squared Error (Minibatch): 70.7494083300835


In [ ]:
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.preprocessing import StandardScaler

batch_size = 32

# Minibatch Optimization для классификатора
perceptron_mb = SGDClassifier(loss='hinge', max_iter=1, warm_start=True)  # warm_start позволяет сохранять веса

for i in range(0, len(X_train_class), batch_size):
    X_batch = X_train_class[i:i + batch_size]
    y_batch = y_train_class[i:i + batch_size]
    perceptron_mb.partial_fit(X_batch, y_batch, classes=np.unique(y_train_class))

# Оценка модели
y_pred_class_mb = perceptron_mb.predict(X_test_class)
print("Classification Report (Minibatch):")
print(classification_report(y_test_class, y_pred_class_mb))

# Minibatch Optimization для регрессора
# Добавим нормализацию данных
scaler = StandardScaler()
X_train_reg_scaled = scaler.fit_transform(X_train_reg)
X_test_reg_scaled = scaler.transform(X_test_reg)

regressor_mb = SGDRegressor(max_iter=1, warm_start=True)

for i in range(0, len(X_train_reg_scaled), batch_size):
    X_batch = X_train_reg_scaled[i:i + batch_size]
    y_batch = y_train_reg[i:i + batch_size]
    regressor_mb.partial_fit(X_batch, y_batch)

# Оценка модели
y_pred_reg_mb = regressor_mb.predict(X_test_reg_scaled)
mse_mb = mean_squared_error(y_test_reg, y_pred_reg_mb)
print(f'Mean Squared Error (Minibatch): {mse_mb}')

Classification Report (Minibatch):
              precision    recall  f1-score   support

           0       0.68      0.97      0.80        29
           1       0.95      0.59      0.73        32

    accuracy                           0.77        61
   macro avg       0.82      0.78      0.77        61
weighted avg       0.82      0.77      0.76        61

Mean Squared Error (Minibatch): 70.79522734730196


Minibatch Optimization для классификатора
- **Minibatch Optimization**: Метод обучения, при котором модель обновляется на основе небольших подмножеств данных (*batch size*).
- **batch_size = 32**: Размер мини-пакета для обучения.
- **SGDClassifier**:
  - **loss='hinge'**: Используется функция потерь "hinge" (подходит для задач классификации).

Итеративное обучение
- Данные разбиваются на мини-пакеты размера **batch_size**.
- **partial_fit**:
  - Выполняет пошаговое обновление модели на каждом мини-пакете.
  - Параметр **classes=np.unique(y_class)** передает все возможные классы для корректного обучения.



## Вывод

В ходе работы были построены модели линейной классификации и регрессии с использованием SGDClassifier и SGDRegressor.
Основные этапы:
- Загрузку данных и их разделение на обучающую и тестовую выборки.
- Настройку гиперпараметров через GridSearchCV для выбора оптимальных функций потерь и регуляризации.
- Реализацию минипакетной оптимизации с использованием метода partial_fit.

Итоговая оценка показала:
- Для классификации были получены лучшие гиперпараметры и достигнуты приемлемые метрики на тестовой выборке.
- Для регрессии была рассчитана среднеквадратичная ошибка (MSE), отражающая качество предсказаний.
- Минипакетная оптимизация продемонстрировала возможность эффективного обучения на частях данных.
